# Asyncio - introduction
___
* Sequential running - The next line of code runs as soon as the previous one has finished, and only one thing is happening at once.
* Asynchronic running - method that helps to optimize tasks to increase perfomance, speed by separating these tasks and completing in special way. 
___

## Main concepts:
* Concurrency(конкурентность) - tasks are happening at the same time on timeline but only one task is active at the moment. We can jump(context switch) between tasks.
* Parallelism(параллелизм) -  tasks are happening __only__ at the same time, like several active concepts at the moment.
    * Main difference: We can work concurrenlty with only 1 core, like changing between tasks.
We need at least 2 cores to be able to use parallelism. Parallelism implies concurrency but it does not always work vice-versa.
* Preemptive multitasking - jumping between tasks in order to minimize duration of execution, we do not set these points but OS. 
* Cooperative multitasking - we create special points where we use methods of async coding, context switch.
___

## Main key words:
async, await, 
single-threaded event loop
* I/O bound - tasks that depends on speed of handling data(waiting responce).
* CPU bound - tasks that directly depends on CPU(calculate pi).
___


#  Understanding processes, threads, multithreading, and multiprocessing
* A process is an application run that has a memory space that other applications cannot
access
* Thread - the smallest construct that can be managed by an operating system.
___
A process will always have at least one thread associated with it, usually known as the main thread. A process
can also create other threads, which are more commonly known as worker or background threads.

In [3]:
# Example
import threading
import os

print(f'Python process running with process id: {os.getpid()}')

total_threads = threading.active_count()
thread_name = threading.current_thread().name

print(f'Python is currently running {total_threads} thread(s)')
print(f'The current thread is {thread_name}')

Python process running with process id: 6364
Python is currently running 6 thread(s)
The current thread is MainThread


* Multithreading - multiple threads are working concurrently.
* Multiprocessing - multiple processes are working concurrently.

In [4]:
def hello_from_thread():
    print(f'Hello from thread {threading.current_thread()}!')

hello_thread = threading.Thread(target=hello_from_thread)
hello_thread.start()

total_threads = threading.active_count()
thread_name = threading.current_thread().name

print(f'Python is currently running {total_threads} thread(s)')
print(f'The current thread is {thread_name}')

# .join - 
hello_thread.join()

Hello from thread <Thread(Thread-5 (hello_from_thread), started 2792)>!Python is currently running 7 thread(s)
The current thread is MainThread



Multithreading is only useful for I/O-bound work because
we are limited by the __global interpreter lock__(GIL) in python.
___
GIL prevents one Python process from
executing more than one Python bytecode instruction at any given time. It is only works for threads but not on processes.
The conflict with threads arises in that the implementation in CPython is not __thread
safe__. The threads are in __race condition__, it can lead to unpredictable behaviour.

In [5]:
import multiprocessing

def hello_from_process():
    print(f'Hello from child process {os.getpid()}!')

if __name__ == '__main__':
    hello_process = multiprocessing.Process(target=hello_from_process)
    hello_process.start()
    print(f'Hello from parent process {os.getpid()}')
    hello_process.join()


Hello from parent process 6364


Working with processes is heavier than working with threads.

The global interpreter lock is released when __I/O operations__ happen. Example below:

In [6]:
import time
import requests
def read_example() -> None:
    response = requests.get('https://www.example.com')
    print(response.status_code)
sync_start = time.time()
read_example()
read_example()
sync_end = time.time()
print(f'Running synchronously took {sync_end - sync_start:.4f} seconds.')


200
200
Running synchronously took 1.6578 seconds.


We can see the proof of it below:

In [7]:
thread_1 = threading.Thread(target=read_example)
thread_2 = threading.Thread(target=read_example)

thread_start = time.time()

thread_1.start()
thread_2.start()
print('All threads running!')

thread_1.join()
thread_2.join()
thread_end = time.time()
print(f'Running with threads took {thread_end - thread_start:.4f} seconds.')

All threads running!
200
200
Running with threads took 0.7985 seconds.


### Single-threaded concurrency
* A __socket__ is a low-level abstraction for sending and receiving data over a network. Sockets support two main
operations: sending bytes and receiving bytes. We write bytes to a socket, which will
then get sent to a remote address, typically some type of server. Sockets are blocking by default. Simply put, this means that when we are waiting for a
server to reply with data, we halt our application or block it until we get data to read.

* An __event loop__ is at the heart of every asyncio application. Event loops are a fairly common design pattern in many systems and have existed for quite some time. 
In asyncio, the event loop keeps a queue of tasks instead of messages. Tasks are wrappers around a coroutine. A coroutine can pause execution when it hits an I/O-bound
operation and will let the event loop run other tasks that are not waiting for I/O operations to complete.

___
* Coroutine - function that wa are able to pause when it encounters an operation that could take a while to
complete.
* The async keyword will let us define a coroutine; the await keyword will let us pause our coroutine when we have a long-running operation.


In [8]:
import asyncio
# We can create coroutine using async before def
async def my_coroutine():
    print("Hello world!")
    

# This function will not work as normal function, it will return it as object and not execute it. We need event loop to execute it.
my_coroutine()

<coroutine object my_coroutine at 0x000001BD4AF3B100>

In [9]:
async def add_one_coroutine(n):
    return n + 1

def add_one(n):
    return n + 1

# print("Simple function: {}".format(type(add_one(1))))
# print("Courotine :{}".format(type(add_one_coroutine(1))))

async def main():
    one = await add_one_coroutine(1)
    two = await add_one_coroutine(2)
    print(one, two)

# In jupiter I have already running event loop so I can just await main() but usually we need to add coroutine to event loop by this commend below 
"asyncio.run(main())"
    
await main()

2 3


asyncio.sleep(n) - stop coroutine for a given time(in seconds)

In [10]:
async def hello_world():
    await asyncio.sleep(1) # forced delay
    return "Hello world"

async def main():
    message = await hello_world()
    print(message)

# We have 1 second to do other python code
await main()

Hello world


In [11]:
# Template, I created it also in util/delay_functions.py
async def delay(delay_seconds):
    print(f"Sleep for {delay_seconds} s")
    await asyncio.sleep(delay_seconds)
    print(f"Woke up after {delay_seconds} s")
    return delay_seconds

# This code below is not working as expected/concurrently, we need to fix it
async def add_one(number: int) -> int:
    return number + 1

async def hello_world_message() -> str:
    await delay(1)
    return "Hello World!"

async def main() -> None:
    message = await hello_world_message()
    one_plus_one = await add_one(1)
    print(one_plus_one)
    print(message)

await main()

Sleep for 1 s
Woke up after 1 s
2
Hello World!


Tasks are wrappers around a coroutine that schedule a coroutine to run on the
event loop as soon as possible.

In [13]:
# creating task
"asyncio.create_task"

async def main():
    sleep_for_three = asyncio.create_task(delay(3))
    print(type(sleep_for_three))
    result = await sleep_for_three
    print(result)

await main()

<class '_asyncio.Task'>
Sleep for 3 s
Woke up after 3 s
3


In [20]:
async def main_without_task():
    sleep_for_three = delay(3)
    sleep_for_two = delay(2)
    sleep_for_one = delay(1)
    
    await sleep_for_three
    await sleep_for_two
    await sleep_for_one

async def main_with_task():
    sleep_for_three = asyncio.create_task(delay(3))
    sleep_for_two = asyncio.create_task(delay(2))
    sleep_for_one = asyncio.create_task(delay(1))
    
    await sleep_for_three
    await sleep_for_two
    await sleep_for_one

# This code works like we are executing simple code in python without concurrency
print("Without tasks:")
await main_without_task()

print('\n')

# This code below works concurrently because now we wrapped our functions with tasks that force them to execute as soon as possible
print("With tasks:")
await main_with_task()

Without tasks:
Sleep for 3 s
Woke up after 3 s
Sleep for 2 s
Woke up after 2 s
Sleep for 1 s
Woke up after 1 s


With tasks:
Sleep for 3 s
Sleep for 2 s
Sleep for 1 s
Woke up after 1 s
Woke up after 2 s
Woke up after 3 s


In [24]:
async def hello_every_second():
    # We can execute this function while waiting when other coroutines finished
    for i in range(2):
        await asyncio.sleep(1)
        print("I'm running other code while I'm waiting!")

async def main():
    sleep_for_three = asyncio.create_task(delay(3))
    sleep_for_three1 = asyncio.create_task(delay(3))

    await hello_every_second()
    await sleep_for_three
    await sleep_for_three1

await main()

Sleep for 3 s
Sleep for 3 s
I'm running other code while I'm waiting!
I'm running other code while I'm waiting!
Woke up after 3 s
Woke up after 3 s


If we have tasks coroutines that takes too long time, we can remove them.
____
Canceling a task is straightforward. Each task object has a method named __cancel__,
which we can call whenever we’d like to stop a task. Canceling a task will cause that
task to raise a __CancelledError__ when we await it, which we can then handle as
needed.

In [28]:
from asyncio import CancelledError

async def main():
    long_task = asyncio.create_task(delay(10))

    seconde_elapsed = 0

    while not long_task.done():
        print("Task not finished, checking again in a second.")
        await asyncio.sleep(1)
        seconde_elapsed += 1
        if seconde_elapsed == 5:
            long_task.cancel()
    

    try:
        await long_task
    except CancelledError:
        print("Your task is cancelled")

await main()

Task not finished, checking again in a second.
Sleep for 10 s
Task not finished, checking again in a second.
Task not finished, checking again in a second.
Task not finished, checking again in a second.
Task not finished, checking again in a second.
Task not finished, checking again in a second.
Your task is cancelled


We can timeout in python using __asyncio.wait_for__

In [29]:
async def main():
    task = asyncio.create_task(delay(2))

    try:
        result = await asyncio.wait_for(task, timeout=1)
        print(result)
    except asyncio.exceptions.TimeoutError:
        print("Time-out")
        print(f"Coroutine was cancelled? {task.cancelled()}")

await main()

Sleep for 2 s
Time-out
Coroutine was cancelled? True


 __asyncio.shield__ function. This function
will prevent cancellation of the coroutine we pass in, giving it a “shield,” which cancellation requests then ignore.

In [30]:
async def main():
    task = asyncio.create_task(delay(10))

    try:
        res = await asyncio.wait_for(asyncio.shield(task), 5)
        print(res)
    except TimeoutError:
        print("Task took longer than five seconds, it will finish soon!")
        res = await task
        print(res)

await main()  

Sleep for 10 s
Task took longer than five seconds, it will finish soon!
Woke up after 10 s
10


* A future is a Python object that contains a single value that you expect to get at some
point in the future but may not yet have.


In [33]:
my_future = asyncio.Future()

print(f"Is my_future done? {my_future.done()}")

my_future.set_result(42)
print(f'Is my_future done? {my_future.done()}')
print(f'What is the result of my_future? {my_future.result()}')

Is my_future done? False
Is my_future done? True
What is the result of my_future? 42


In [34]:
def make_request():
    future = asyncio.Future()
    asyncio.create_task(set_future_value(future))
    return future

async def set_future_value(future):
    await asyncio.sleep(1)
    future.set_result(42)

async def main():
    future = make_request()
    print(f"Is request done? {future.done()}")
    value = await future
    print(f"Is request done? {future.done()}")
    print(value)

await main()

Is request done? False
Is request done? True
42


Let`s make our own decorator __async_timed__ for detecting execution time of out coroutines, tasks

In [37]:
import functools
from typing import Callable, Any

def async_timed():
    def wrapper(func: Callable):
        @functools.wraps(func)
        async def wrapped(*args, **kwargs):
            print(f"starting {func} with args {args} {kwargs}'")
            start = time.time()
            try:
                return await func(*args, **kwargs)
            finally:
                end = time.time()
                total = end - start
                print(f"finished {func} in {total:.4f} second(s)")
        return wrapped

    return wrapper

# Example of usage

@async_timed()
async def main():
    task1 = asyncio.create_task(delay(3))
    task2 = asyncio.create_task(delay(2))

    await task1
    await task2

await main()

starting <function main at 0x000001BD4AF43920> with args () {}'
Sleep for 3 s
Sleep for 2 s
Woke up after 2 s
Woke up after 3 s
finished <function main at 0x000001BD4AF43920> in 3.0030 second(s)


We need to be carefull because sometime using cocurrent methods can reduce speed of execution instead of increasing
___
* Trying to add concurrency to CPU bound tasks without multiprocessing is huge error.
* Using blocking I/O-bound APIs without using multithreading is huge error.
___
Because of GIL we can only solve CPU bound tasks only single-thread-concurrently.

In [38]:
# It is bad example, we should not use concurrency everywhere, especially when we have CPU bound and I/O bound together

@async_timed()
async def cpu_bound_work():
    counter = 0
    for i in range(10000000):
        counter = counter + 1
    return counter

@async_timed()
async def main():
    task1 = asyncio.create_task(cpu_bound_work())
    task2 = asyncio.create_task(cpu_bound_work())
    task3 = asyncio.create_task(delay(4))

    await task1
    await task2
    await task3

# We can see that this code is not executing concurrently because we added concurrency to every task
await main()

starting <function main at 0x000001BD4AF42840> with args () {}'
starting <function cpu_bound_work at 0x000001BD4AF42B60> with args () {}'
finished <function cpu_bound_work at 0x000001BD4AF42B60> in 0.5286 second(s)
starting <function cpu_bound_work at 0x000001BD4AF42B60> with args () {}'
finished <function cpu_bound_work at 0x000001BD4AF42B60> in 0.5495 second(s)
Sleep for 4 s
Woke up after 4 s
finished <function main at 0x000001BD4AF42840> in 5.0936 second(s)


Any function that performs I/O that is not a coroutine or performs time-consuming CPU
operations can be considered blocking. 

In [40]:
@async_timed()
async def get_example_status():
    # Code is not working concurrently because request.get is blocking I/O bound task
    return requests.get('http://www.example.com').status_code

@async_timed()
async def main():
    task1 = asyncio.create_task(get_example_status())
    task2 = asyncio.create_task(get_example_status())
    task3 = asyncio.create_task(get_example_status())

    await task1
    await task2
    await task3

# We can solve this problem by using 'aoihttp' or force asyncio to use multithreading
await main()

starting <function main at 0x000001BD4B640220> with args () {}'
starting <function get_example_status at 0x000001BD4B642A20> with args () {}'
finished <function get_example_status at 0x000001BD4B642A20> in 0.3839 second(s)
starting <function get_example_status at 0x000001BD4B642A20> with args () {}'
finished <function get_example_status at 0x000001BD4B642A20> in 0.3735 second(s)
starting <function get_example_status at 0x000001BD4B642A20> with args () {}'
finished <function get_example_status at 0x000001BD4B642A20> in 0.3759 second(s)
finished <function main at 0x000001BD4B640220> in 1.1333 second(s)


### Accessing and manually managing the event loop
It can be very useful if we want special behaviour for our event loop or if we want to debug.

In [52]:
@async_timed()
async def main():
    await asyncio.sleep(1)

loop = asyncio.new_event_loop()

# We need to close event loop even if we get exception
# Usually we do not have running event loop but in jypiter I have already running event loop so I can not do below code
"""
try:
    loop.run_until_complete(main())
finally:    
    loop.close()
"""

current_loop = asyncio.get_running_loop()
print(f"type = {type(current_loop)}, {current_loop}")
await main()

type = <class 'asyncio.windows_events._WindowsSelectorEventLoop'>, <_WindowsSelectorEventLoop running=True closed=False debug=False>
starting <function main at 0x000001BD4C1ADF80> with args () {}'
finished <function main at 0x000001BD4C1ADF80> in 1.0077 second(s)


In [56]:
def call_later():
    print("I'm being called in the future!")

@async_timed()
async def main():
    loop = asyncio.get_running_loop()
    loop.call_soon(call_later)
    await delay(1)

await main()

starting <function main at 0x000001BD4AF43BA0> with args () {}'
Sleep for 1 s
I'm being called in the future!
Woke up after 1 s
finished <function main at 0x000001BD4AF43BA0> in 1.0004 second(s)


In [62]:
from IPython.display import clear_output

# Using debug mode 
"asyncio.run(coroutine, debug=True)"

# Using debug mode if we have running loop
current_loop = asyncio.get_running_loop()
current_loop.set_debug(True)
print(current_loop)

# Changing threshold to detect slow tasks, coroutines
current_loop.slow_callback_duration = .250

@async_timed()
async def cpu_bound_work():
    counter = 0
    for i in range(10000000):
        counter = counter + 1
    return counter

@async_timed()
async def main():
    task1 = asyncio.create_task(cpu_bound_work())
    task2 = asyncio.create_task(delay(4))

    await task1
    await task2

await main()
clear_output()